In [1]:
%load_ext sql

There's a new jupysql version available (0.10.10), you're running 0.10.8. To upgrade: pip install jupysql --upgrade
Deploy AI and data apps for free on Ploomber Cloud! Learn more: https://docs.cloud.ploomber.io/en/latest/quickstart/signup.html


Loading configurations from /home/hugo/Documents/skola/edaf75/repo/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True


In [2]:
%sql sqlite:///movies.sqlite

Connecting to 'sqlite:///movies.sqlite'

In [52]:
%%sql
PRAGMA foreign_keys=OFF;

DROP TABLE IF EXISTS theaters;
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS movies;
DROP TABLE IF EXISTS performances;
DROP TABLE IF EXISTS tickets;

PRAGMA foreign_keys=ON;

-- Create the tables.
CREATE TABLE theaters (
    theater_name    TEXT,
    capacity        INT,
    PRIMARY KEY     (theater_name)
);

CREATE TABLE customers (
    user_name       TEXT,
    full_name       TEXT,
    password        TEXT,
    PRIMARY KEY     (user_name)
);

CREATE TABLE movies (
    imdb            TEXT,
    title           TEXT,
    year            INT,
    PRIMARY KEY     (imdb)
);

CREATE TABLE performances (
    performance_id  TEXT DEFAULT (lower(hex(randomblob(16)))),
    start_time      TIME,
    date            DATE,
    theater_name    TEXT,
    imdb            TEXT,
    PRIMARY KEY     (performance_id),
    FOREIGN KEY     (theater_name) REFERENCES theaters(theater_name),
    FOREIGN KEY     (imdb) REFERENCES movies(imdb)          
);

CREATE TABLE tickets (
    ticket_id       TEXT DEFAULT (lower(hex(randomblob(16)))),
    user_name       TEXT,
    performance_id  TEXT,
    PRIMARY KEY     (ticket_id),
    FOREIGN KEY     (user_name) REFERENCES customers(user_name),
    FOREIGN KEY     (performance_id) REFERENCES performances(performance_id)
);

drop trigger if exists ticket_limit;
    create trigger ticket_limit
        before insert on tickets
        when
            (with taken_seats(performance_id, taken) as (
                select performance_id, count() as taken
                from performances
                        join tickets using(performance_id)
                group by performance_id
            )
            select capacity - coalesce(taken, 0) as remaining_seats
            from performances
                    join theaters using(theater_name)
                    join movies using(imdb)
                    left outer join taken_seats using(performance_id)
            where performance_id = NEW.performance_id) <= 0
        begin
            select raise (rollback, "No tickets left");
        end;

-- Insert data into the tables.
INSERT
INTO    theaters(theater_name, capacity)
VALUES  ('Kino', 10),
        ('Regal', 16),
        ('Skandia', 100);

insert
into    movies(imdb, title, year)
values  ('abc', 'Forrest Gump', 1993);

insert
into    performances(performance_id, start_time, date, theater_name, imdb)
values  ('perf_1', '19:00', '2024-02-14', 'Kino', 'abc');

insert
into    customers(user_name, full_name, password)
values  ('bob', 'Bob', 'bob');

Running query in 'sqlite:///movies.sqlite'

3 rows affected.

1 rows affected.

1 rows affected.

1 rows affected.

""


In [53]:
%%sql
select *
from performances

Running query in 'sqlite:///movies.sqlite'

,performance_id,start_time,date,theater_name,imdb
0,perf_1,19:00,2024-02-14,Kino,abc


In [56]:
%%sql
insert
into   tickets(user_name, performance_id)
values ('bob', 'perf_1')
returning ticket_id

Running query in 'sqlite:///movies.sqlite'

,ticket_id
0,ed77ef64de787e8105005725957fe5e8


In [54]:
%%sql
select *
from tickets

Running query in 'sqlite:///movies.sqlite'

,ticket_id,user_name,performance_id


In [57]:
%%sql
with taken_seats(performance_id, taken) as (
    select performance_id, count() as taken
    from performances
            join theaters using(theater_name)
            join tickets using(performance_id)
    group by performance_id
)
select performance_id, date, start_time, title, year, theater_name, capacity - coalesce(taken, 0) as remaining_seats
from performances
        join theaters using(theater_name)
        join movies using(imdb)
     left outer join taken_seats using(performance_id)

Running query in 'sqlite:///movies.sqlite'

,performance_id,date,start_time,title,year,theater_name,remaining_seats
0,perf_1,2024-02-14,19:00,Forrest Gump,1993,Kino,9


In [36]:
%%sql
with taken_seats(performance_id, taken) as (
    select performance_id, count() as taken
    from performances
            join theaters using(theater_name)
            join tickets using(performance_id)
    group by performance_id
)
select (capacity - coalesce(taken, 0))>0 as remaining_seats
from performances
        join theaters using(theater_name)
        join movies using(imdb)
        left outer join taken_seats using(performance_id)

Running query in 'sqlite:///movies.sqlite'

,remaining_seats
0,1


In [37]:
%%sql
with user_tickets(performance_id, count) as (
    select   performance_id, count() as count
    from     tickets
    where    user_name = 'bob'
    group by performance_id
)
select date, start_time, theater_name, title, year, count as nbrOfTickets
from   user_tickets
        join performances using(performance_id)
        join movies using(imdb)

Running query in 'sqlite:///movies.sqlite'

,date,start_time,theater_name,title,year,nbrOfTickets
0,2024-02-14,19:00,Kino,Forrest Gump,1993,1
